In [ ]:
from ultralytics import YOLO
import os
import numpy as np
from tqdm import tqdm

In [ ]:
model = YOLO('runs/detect/train8/weights/best.pt')
set = 'val'

In [ ]:
def count_objects_in_label_file(label_file):
    try:
        with open(label_file, 'r') as f:
            lines = f.readlines()
        
        # Remove comments and empty lines
        lines = [line.strip() for line in lines if line.strip() and not line.strip().startswith('#')]
        
        # Count the number of lines (objects)
        num_objects = len(lines)
        
        return num_objects
    except Exception as e:
        print(f"Error counting objects in label file: {e}")
        return 0

In [ ]:
# Get folder
train_image_folder = f"/home/ucloud/data_yolo/images/train"
train_labels_folder = f"/home/ucloud/data_yolo/labels/train"

val_image_folder = f"/home/ucloud/data_yolo/images/val"
val_labels_folder = f"/home/ucloud/data_yolo/labels/val"

# Ids
train_ids_path = '/home/ucloud/train.txt'
val_ids_path = '/home/ucloud/validation.txt'

# Train
with open(train_ids_path, 'r') as file:
    train_ids = [line.strip() for line in file]
# Val
with open(val_ids_path, 'r') as file:
    val_ids = [line.strip() for line in file]
    
train_image_paths = [os.path.join(train_image_folder, f"{id}.jpg") for id in train_ids]
val_image_paths = [os.path.join(val_image_folder, f"{id}.jpg") for id in val_ids]

train_label_paths = [os.path.join(train_labels_folder, f"{id}.txt") for id in train_ids]
val_label_paths = [os.path.join(val_labels_folder, f"{id}.txt") for id in val_ids]

In [ ]:
# Set path

if set == 'train':
    images_path = train_image_paths
    labels_path = train_label_paths
elif set == 'val':
    images_path = val_image_paths
    labels_path = val_label_paths

In [ ]:
# Assuming images_path is a list of image paths
batch_size = 20

true_num = []
pred_num = []

# Calculate the total number of batches
num_batches = (len(images_path) + batch_size - 1) // batch_size

# Process images in batches
for batch_idx in tqdm(range(num_batches), desc='Processing Batches'):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(images_path))
    batch_images_path = images_path[start_idx:end_idx]
    batch_label_path = labels_path[start_idx:end_idx]

    # Process the current batch
    batch_results = []
    for image_path in batch_images_path:
        result = model(image_path)
        batch_results.append(len(result[0].boxes.cls))
    pred_num.extend(batch_results)
    
    batch_results = []
    for label_path in batch_label_path:
        result = count_objects_in_label_file(label_path)
        batch_results.append(result)
    true_num.extend(batch_results)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_log_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
true_labels= true_num
predicted_labels = pred_num

In [ ]:
print("Accuracy multi-labal:", accuracy_score(true_labels, predicted_labels))
print("Mean Squared Logarithmic Error:", mean_squared_log_error(true_labels, predicted_labels))

In [ ]:
# Create a histogram plot using seaborn
sns.histplot([x for x in true_labels if x != 0], bins=50, kde=True)  # Adjust the number of bins as needed
plt.title('Histogram Plot')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a histogram plot using seaborn
sns.histplot([x for x in predicted_labels if x != 0], bins=50, kde=True)  # Adjust the number of bins as needed
plt.title('Histogram Plot')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(true_labels, predicted_labels, color='blue', alpha=0.5)  # Adjust alpha for transparency
plt.title('Scatter Plot of Predicted vs. True Labels')
plt.xlabel('True Labels')
plt.ylabel('Predicted Labels')
plt.grid(True)
plt.show()

In [ ]:
# Create a violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(data=[true_labels, predicted_labels], palette=['blue', 'red'])
plt.title('Violin Plot of Predicted vs. True Labels')
plt.xlabel('Labels')
plt.ylabel('Value')
plt.grid(True)
plt.show()

In [ ]:
true_labels_binary = [0 if label == 0 else 1 for label in true_labels]
predicted_labels_binary = [0 if label == 0 else 1 for label in predicted_labels]

accuracy = accuracy_score(true_labels_binary, predicted_labels_binary)
precision = precision_score(true_labels_binary, predicted_labels_binary)
recall = recall_score(true_labels_binary, predicted_labels_binary)
f1 = f1_score(true_labels_binary, predicted_labels_binary)
cm = confusion_matrix(true_labels_binary, predicted_labels_binary)

In [ ]:
# Define class labels
classes = ['Empty', 'Contaminated']
conf_matrix = cm

# Create a new figure
plt.figure(figsize=(8, 6))

# Plot the confusion matrix
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)

# Add colorbar
plt.colorbar()

# Add tick marks and labels
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

# Add text annotations
thresh = conf_matrix.max() / 2.
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

# Add labels and title
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)